# This notebook provides model runs for the PAMIR dataset

## It generates figures 4, 6b and 9

In [1]:
# Import statements
%matplotlib widget
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.lines import Line2D
from matplotlib import gridspec


# local SMRT import
from smrt import make_snowpack, make_model, sensor, make_atmosphere
from smrt.substrate.reflector import make_reflector
from smrt.core.globalconstants import DENSITY_OF_ICE, FREEZING_POINT

# Evaluation paper import
from common_functions import symmetrize_microstructure, microstructure_colour_list, microstructure_short_labels, me, rmse

### Read in field data

In [2]:
# Microstructure
df_pamir09 = symmetrize_microstructure('data/PAMIR/1984May9sections_EMMS_acf_parameters_v1.0.csv')
df_pamir10 = symmetrize_microstructure('data/PAMIR/1984May10sections_EMMS_acf_parameters_v1.0.csv')

In [3]:
# Layer thickness
def add_pamir_layer_thickness(df, depths):
    # This calculates maximum layer thicknesses from data handwritten in image files
    # Calculates thickness from layer boundaries
    df['layer_depth'] = pd.Series(depths)
    df['layer_thickness'] = df.layer_depth.diff()
    df.loc[0,'layer_thickness'] = depths[0] # Diff gives NaN for 1st layer, calcs difference for others
    return df

# Specify layer depths from 1984May9sections.jpg and 1984May10sections.jpg files
layer_depths09 = [3.60e-2, 6.66e-2, 9.62e-2, 12.63e-2, 15.5e-2]
layer_depths10 = [3.06e-2, 5.97e-2, 8.90e-2, 12.06e-2, 15.12e-2]

df_pamir09 = add_pamir_layer_thickness(df_pamir09, layer_depths09)
df_pamir10 = add_pamir_layer_thickness(df_pamir10, layer_depths10)

In [4]:
# Read general datafile - contains TB, refrozen depth etc. as a function of time
df_obs = pd.read_csv('data/PAMIR/PAMIR_obs_May84.csv')
# Need to change the name of the T10H column to T10.4H
df_obs.rename(columns={'T10H':'T10.4H'}, inplace=True)
# Remove all observations where refrozen depth is zero (not simulating these and need arrays same length)
df_obs = df_obs[df_obs['d(m)'] > 0]

## Plot Figure 4 - change in temperature, brightness temperature and refrozen layer depth

In [5]:
plt.close()
fig = plt.figure()
spec = gridspec.GridSpec(ncols=1, nrows=2,
                         height_ratios=[2, 1])

ax1 = fig.add_subplot(spec[0])
ax2 = fig.add_subplot(spec[1])

# Split two refreezing periods
meltpoint = 21
ax2.plot(df_obs['Time(h)'][:meltpoint], df_obs['d(m)'][:meltpoint], 'bo-', linewidth=4)
ax2.plot(df_obs['Time(h)'][meltpoint:], df_obs['d(m)'][meltpoint:], 'bo-', linewidth=4)

ax2.set_xlabel('Time (h)')
# Make the y-axis label and tick labels match the line color.
ax2.set_ylabel('Refrozen depth (m)')

ax1.plot(df_obs['Time(h)'], df_obs['T35V'], 'r^', label='TB35V')
ax1.plot(df_obs['Time(h)'], df_obs['Ts(C)'] + FREEZING_POINT, 'r-.', label='Surface T')
ax1.set_xticks([])
ax1.set_ylabel('Temperature (K)')

#plt.title('PAMIR freeze-melt-freeze experiment May 1984')
ax1.legend(loc='best')
fig.canvas.layout.width = '600px'

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Set up simulation configuration

In [6]:
# PAMIR radiometers
rad = sensor.passive([4.9e9, 10.4e9, 21e9, 35e9, 94e9], 50)

In [7]:
# Set up model
model = make_model("iba","dort")

### Look at atmosphere

In [8]:
# Uncomment to look at time-dependency of atmospheric TB
#plt.close()
#df_obs.plot('Time(h)', ['Ts4.9', 'Ts10', 'Ts21', 'Ts35', 'Ts94'], marker='o')

### Make list of snowpacks

In [9]:
# Create empty array for list of snowpacks
all_snowpacks = []
# Store all times where some snow is frozen
time_list = df_obs[df_obs['d(m)']>0]['Time(h)']
# Substrate: is melting snow
absorber = make_reflector(specular_reflection=0.04, temperature=FREEZING_POINT)

# Loop over time (rows in df_obs)
for index, time in df_obs.iterrows():
    if (time['Time(h)'] < 40) and (time['d(m)'] > 0.):
        # Use 9th May data for first freeze cycle
        df = df_pamir09
    else:
        # Use 10th May for second freeze cycle
        df = df_pamir10
        
    # Calculate number of layers (number of full layers plus a partial layer)
    nlayers = (df.layer_depth < time['d(m)']).sum() + 1
    nlayers = np.clip(nlayers, 0, 5)  # Maximum of 5 layers: bottom layer will be extended beyond microstructure observations
    if time['d(m)'] > 0.:
        # Some snow is frozen
        thickness = df.layer_thickness[0:nlayers]
        thickness[nlayers-1] = time['d(m)'] - df.layer_thickness[0:nlayers-1].sum()  # NB referring to index [-1] causes overwrite in df09.depth
        density = df['density'][0:nlayers]
        d_shs = df['d_shs'][0:nlayers]
        tau = df['tau'][0:nlayers]
        # y-component assumed to be same as x-component
        l_ex = df['l_ex'][0:nlayers]
        d_sph = df['d_sph'][0:nlayers]
        xi_ts = df['xi_ts'][0:nlayers]
        domain_ts = df['domain_ts'][0:nlayers]
        xi_grf = df['xi_grf'][0:nlayers]
        domain_grf = df['domain_grf'][0:nlayers]
        temperature_half_interval = 0.5 * time['Ts(C)'] / nlayers
        temperature_in_c = np.linspace(time['Ts(C)'] - temperature_half_interval, temperature_half_interval, nlayers)
        temperature = temperature_in_c + FREEZING_POINT
        
        # Atmosphere
        tbdown = {4.9e9 : time['Ts4.9'], 10.4e9 : time['Ts10'], 21e9 : time['Ts21'],
          35e9 : time['Ts35'], 94e9 : time['Ts94']}
        atmos = make_atmosphere('simple_isotropic_atmosphere', tbdown=tbdown)
        
        # Set up snowpacks
        snowpack_shs = atmos + make_snowpack(thickness=thickness,
                         microstructure_model="sticky_hard_spheres",
                         temperature=temperature,
                         density=density,
                         radius=(d_shs / 2.),
                         stickiness=tau,
                         substrate=absorber)
        snowpack_exp = atmos + make_snowpack(thickness=thickness,
                         microstructure_model="exponential",
                         temperature=temperature,
                         density=density,
                         corr_length=l_ex,
                         substrate=absorber)
        snowpack_ind = atmos + make_snowpack(thickness=thickness,
                         microstructure_model="independent_sphere",
                         temperature=temperature,
                         density=density,
                         radius=(d_sph / 2.),
                         substrate=absorber)
        snowpack_ts = atmos + make_snowpack(thickness=thickness,
                         microstructure_model="teubner_strey",
                         temperature=temperature,
                         density=density,
                         corr_length=xi_ts,
                         repeat_distance=domain_ts,
                         substrate=absorber)
        snowpack_grf = atmos + make_snowpack(thickness=thickness,
                         microstructure_model="gaussian_random_field",
                         temperature=temperature,
                         density=density,
                         corr_length=xi_grf,
                         repeat_distance=domain_grf,
                         substrate=absorber)
        all_snowpacks.append([snowpack_exp, snowpack_shs, snowpack_ind, snowpack_ts, snowpack_grf])

# Collect labels for snowpacks: label by microstructure model
snowpack_labels = [all_snowpacks[0][sp].layers[0].microstructure_model.__name__ for sp in range(len(all_snowpacks[0]))]

### Run model

In [10]:
result = model.run(rad, all_snowpacks, snowpack_dimension=('time', time_list))

In [11]:
# Rename snowpacks with microstructure list
result.coords['snowpack'] = snowpack_labels

### Plot results - Figure 6b

In [12]:
fig, (axv, axh) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(6,12))
plt.rcParams.update({'font.size': 16})

def get_obs_labels(f, pol):
    # Generate column names to refer to observation data frame
    # Will be of format e.g. 'T4.9V'
    f_ghz = f / 1e9
    if f_ghz.is_integer(): f_ghz = int(f_ghz)
    # Generate obs label
    return ('T' + str(f_ghz) + pol)
    
    
# Loop over frequencies
for f in rad.frequency:

    # Get column names for observation dataframe
    v_string = get_obs_labels(f, 'V')
    h_string = get_obs_labels(f, 'H')
   
    # Plot V pol for each microstructure model
    [axv.scatter(df_obs[v_string], result.TbV(snowpack=m, frequency=f), c=microstructure_colour_list()[m],
                  label=microstructure_short_labels()[m], marker='^') for m in snowpack_labels]
     # Plot H pol for each microstructure model
    [axh.scatter(df_obs[h_string], result.TbH(snowpack=m, frequency=f), c=microstructure_colour_list()[m],
                  marker='H', alpha=0.2) for m in snowpack_labels]   


#1-1 line
x = np.linspace(100, 270, 10)
xticks = np.arange(125, 280, 25)
for ax in [axv, axh]:
    ax.plot(x,x,'k--', alpha=0.3)
    ax.set_xticks(xticks)
    ax.set_yticks(xticks)
    ax.set_xlim([110, 275])
    ax.set_ylim([110, 275])
    
plt.xlabel('Observed TB (K)')
#plt.ylabel('Simulated TB (K)')
#plt.title('PAMIR')
plt.tight_layout()
plt.show()

fig.canvas.layout.width = '600px'
fig.canvas.layout.height = '600px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Calculate Error Statistics

In [13]:
# V-pol RMSE
# Make time the index: need to make sure coords are same as for simulations
obs = df_obs.to_xarray().set_index(index='Time(h)').rename({'index':'time'})

# Make frequency a dimension - want in same format as results so can calculate ME / RMSE
obs.coords['frequency'] = rad.frequency
# Extract just H-pol data
Hobs = xr.concat([obs['T4.9H'], obs['T10.4H'], obs['T21H'], obs['T35H'], obs['T94H']], dim='frequency')
Hobs.coords['frequency'] = rad.frequency
# Extract just V-pol data
Vobs = xr.concat([obs['T4.9V'], obs['T10.4V'], obs['T21V'], obs['T35V'], obs['T94V']], dim='frequency')
Vobs.coords['frequency'] = rad.frequency

In [14]:
# ME (H, V)
print ('\nMean Error')
[print (microstructure_short_labels()[m], np.round(me(Hobs, result.TbH(snowpack=m)),1), 
        np.round(me(Vobs, result.TbV(snowpack=m)),1)) for m in snowpack_labels]


Mean Error
EXP 6.4 7.1
SHS 18.8 18.9
IND 5.3 6.0
TS 3.3 4.1
GRF 2.1 2.8


[None, None, None, None, None]

In [15]:
# RMSE (H, V)
print ('\nRoot Mean Squared Error')
[print (microstructure_short_labels()[m], np.round(rmse(Hobs, result.TbH(snowpack=m)),1), 
        np.round(rmse(Vobs, result.TbV(snowpack=m)),1)) for m in snowpack_labels]


Root Mean Squared Error
EXP 15.1 16.4
SHS 35.1 37.0
IND 14.2 14.9
TS 11.7 12.0
GRF 10.4 10.2


[None, None, None, None, None]

## Plot time series - figure 9

In [16]:
def plot_timeseries(f, pol):
    # Split refreezing periods
    mp1 = 21
    mp2 = 26

    # Get column names for observation dataframe
    obs_string = get_obs_labels(f, pol)
    # Extract res
    
    # Set plot characteristics
    ymin = 100
    ymax = 275
    if pol == 'V':  
        alf = 0.8 # Alpha
        symb = '^'
        plt.yticks(np.arange(ymin, ymax, step=50))
        plt.ylabel(str(f / 1e9) + ' GHz ' + 'T$_B$ [K]')
        ls = '-' # Line style
    else:
        alf = 0.4
        symb = 'H'
        plt.yticks(np.arange(ymin, ymax, step=50))
        plt.gca().axes.yaxis.set_ticklabels([])
        ls = '--'
    ms = 10 # Marker size (observations)
    plt.ylim((100,275))
    
    # V-pol
    # Period 1
    plt.plot(df_obs['Time(h)'][:mp1], df_obs[obs_string][:mp1], c='k', alpha = alf, label='Vobs', marker=symb, markersize=ms, linestyle=ls)
    [plt.plot(df_obs['Time(h)'][:mp1], result.Tb(snowpack=m, frequency=f, polarization=pol)[:mp1], alpha=alf, linestyle=ls,
             c=microstructure_colour_list()[m], label=microstructure_short_labels()[m], marker=symb) for m in snowpack_labels]
   # Period 2
    plt.plot(df_obs['Time(h)'][mp1:mp2],df_obs[obs_string][mp1:mp2], c='k', alpha = alf, label='Vobs', marker=symb, markersize=ms, linestyle=ls)
    [plt.plot(df_obs['Time(h)'][mp1:mp2], result.Tb(snowpack=m, frequency=f, polarization=pol)[mp1:mp2], alpha=alf, linestyle=ls,
             c=microstructure_colour_list()[m], label=microstructure_short_labels()[m], marker=symb) for m in snowpack_labels]
   # Period 3
    plt.plot(df_obs['Time(h)'][mp2:],df_obs[obs_string][mp2:], c='k', alpha = alf, label='Vobs', marker=symb, markersize=ms, linestyle=ls)
    [plt.plot(df_obs['Time(h)'][mp2:], result.Tb(snowpack=m, frequency=f, polarization=pol)[mp2:], alpha=alf, linestyle=ls,
             c=microstructure_colour_list()[m], label=microstructure_short_labels()[m], marker=symb) for m in snowpack_labels]

    # Put legend and title on 4.9 GHz plots
    if f == 4.9e9:
        legend = [Line2D([0], [0], marker=symb, color='k', linestyle=ls, markerfacecolor='k', label='Obs', alpha=alf,
                        markersize=ms)] + [Line2D([0], [0], marker=symb, color=microstructure_colour_list()[m], linestyle=ls,
                        markerfacecolor=microstructure_colour_list()[m], markersize=8,
                        label=microstructure_short_labels()[m], alpha=alf) for m in snowpack_labels]
        plt.legend(handles=legend, loc='lower center', ncol=3)
        plt.title(pol + ' polarization')
        
    # Put x label on 94GHz plots
    if f == 94e9:
        plt.xlabel('Time [hours]')
    

In [17]:
# Plot PAMIR time-series
plt.close()
plt.rcParams.update({'font.size': 12})
fig = plt.figure(constrained_layout=False)
spec = fig.add_gridspec(ncols=2, nrows=5, wspace=0.05, hspace=0.05)

# Col 1: V-pol
for i, freq in enumerate(rad.frequency):
    fig.add_subplot(spec[i,0])
    plot_timeseries(freq, 'V')

# Col 2: H-pol
for i, freq in enumerate(rad.frequency):
    fig.add_subplot(spec[i,1])
    plot_timeseries(freq, 'H')
    
plt.show()

fig.canvas.layout.width = '1000px'
fig.canvas.layout.height = '1200px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Check Error Statistics for two separate refreeze cycles

In [18]:
result1V = xr.concat([result.TbV(time=t) for t in result.time[:21]], dim='time')
result2V = xr.concat([result.TbV(time=t) for t in result.time[21:]], dim='time')

In [19]:
obs1V = xr.concat([Vobs.sel(time=t) for t in Vobs.time[:21]], dim='time')
obs2V = xr.concat([Vobs.sel(time=t) for t in Vobs.time[21:]], dim='time')

In [20]:
# RMSE (H, V)
print ('\nRoot Mean Squared Error - Day 1')
[print (microstructure_short_labels()[m], np.round(rmse(obs1V, result1V.sel(snowpack=m)),1)) for m in snowpack_labels]
print ('\nRoot Mean Squared Error - Day 2')

[print (microstructure_short_labels()[m], np.round(rmse(obs2V, result2V.sel(snowpack=m)),1)) for m in snowpack_labels]


Root Mean Squared Error - Day 1
EXP 17.4
SHS 40.0
IND 15.9
TS 12.5
GRF 10.3

Root Mean Squared Error - Day 2
EXP 13.0
SHS 26.0
IND 11.5
TS 10.3
GRF 10.0


[None, None, None, None, None]